In [1]:
import web3
from web3 import Web3,HTTPProvider
from eth_keys import keys
import time
w3 = Web3(web3.HTTPProvider("http://127.0.0.1:8545"))
w3.isConnected()

True

# check time travel

In [10]:
start = w3.eth.getBlock(w3.eth.blockNumber).timestamp
w3.provider.make_request("evm_increaseTime",[2*24*3600])
time.sleep(6)
end = w3.eth.getBlock(w3.eth.blockNumber).timestamp
end - start

172805

# check accounts

In [3]:
print(w3.eth.accounts)
print(w3.eth.coinbase)
print(Web3.fromWei(w3.eth.getBalance(w3.eth.coinbase),'ether'))

['0x0981D20D34A0FC96e73FfA783D0C560156142d90']
0x0981D20D34A0FC96e73FfA783D0C560156142d90
1000000000000000000000000000000


# another account

In [4]:
test_public_address = "0x9e74086457814b6d7A87da12F000c6A91637784A"
test_private_key = 0xc95c4e45c20601eb1c95d6b18ea4f6ad5ca2f219e94ebd3c499a1d1e5a58ca4d
print(keys.PrivateKey(Web3.toBytes(test_private_key)).public_key.to_checksum_address())
print(Web3.fromWei(w3.eth.getBalance(test_public_address),'ether'))

0x9e74086457814b6d7A87da12F000c6A91637784A
0


# send 1 ether to another account

In [5]:
signed_txn = w3.eth.account.signTransaction(dict(
    nonce=w3.eth.getTransactionCount(w3.eth.coinbase),
    gasPrice = w3.eth.gasPrice, 
    gas = 100000,
    to=test_public_address,
    value=Web3.toWei(1,'ether')
  ),
  Web3.toBytes(0x67f549fdaf5e0173cb71fcc6dd66a19aac0e63c5a09acb287007dd98b0571f51))

tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt)
print(Web3.fromWei(w3.eth.getBalance(test_public_address),'ether'))

AttributeDict({'transactionHash': HexBytes('0x7f8abdaf76b46536cb9ddf0920d73a2be8ddd79aeb54bcb262e409a4a72c12cd'), 'transactionIndex': 0, 'blockHash': HexBytes('0x176b555ee399048d1cb821016b1acd063ce3433fd7b74277539bff5b338185fc'), 'blockNumber': 9, 'from': '0x0981D20D34A0FC96e73FfA783D0C560156142d90', 'to': '0x9e74086457814b6d7A87da12F000c6A91637784A', 'gasUsed': 21000, 'cumulativeGasUsed': 21000, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})
1


# send 10 transactions

In [7]:
tx_hashes = []
nonce = w3.eth.getTransactionCount(w3.eth.coinbase)
for i in range(10):
    signed_txn = w3.eth.account.signTransaction(dict(
        nonce=nonce,
        gasPrice = w3.eth.gasPrice, 
        gas = 100000,
        to=test_public_address,
        value=Web3.toWei(1,'ether')
      ),
      Web3.toBytes(0x67f549fdaf5e0173cb71fcc6dd66a19aac0e63c5a09acb287007dd98b0571f51))

    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    tx_hashes.append(tx_hash)
    nonce+=1
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt)

AttributeDict({'transactionHash': HexBytes('0xa2bc610bee58dbf8c9cd98f724a085a58d3c371105d8119aa0ebb8bcadfabdcf'), 'transactionIndex': 9, 'blockHash': HexBytes('0x47afa2c9ef258746fb79c200fe9196f6f0ac0786695fa5c36f9345d7242be61d'), 'blockNumber': 12, 'from': '0x0981D20D34A0FC96e73FfA783D0C560156142d90', 'to': '0x9e74086457814b6d7A87da12F000c6A91637784A', 'gasUsed': 21000, 'cumulativeGasUsed': 210000, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})


# check 10 transaction blocks

In [8]:
for i in range(10):
    print(w3.eth.getTransactionReceipt(tx_hashes[i]).blockNumber)

12
12
12
12
12
12
12
12
12
12


In [9]:
tx_hashes

[HexBytes('0x17b4797281337e0fba45b807bef9d6b3633a3828e29f58488a5a48ca86374c0e'),
 HexBytes('0x1f12629f096c915b927707bb09696cdd22e0f500a7ac6d241593162099b7a569'),
 HexBytes('0x0c21b51149336a58b00d0de034b4e3bf688c651c568743ac30d17372ffa27fa4'),
 HexBytes('0x4e2120596196fd8be2a71be2e55e841e9b5dadf14a67e99e5e7e7e680ee6a563'),
 HexBytes('0xee0e2fe394df57c729d98b5136202099855b6346d98cbb9f2cc113f52aba251a'),
 HexBytes('0xc614f2b857903151af7f5d78b29b301d275f437da94cff1c7b3b5759baea6577'),
 HexBytes('0xf7d8e2ec32d8147c7cb43b91597b4aa4cb6d382398a53ad8792bb3c9711a7ec5'),
 HexBytes('0x8a0df4838e849c8061ceeb7802748f1834c836027aa02009a9290a37e89a5814'),
 HexBytes('0x3b80bd120d4fcdfb831055ecac8e2495ddc3de0d5380f85f92b9d6f8908d1181'),
 HexBytes('0xa2bc610bee58dbf8c9cd98f724a085a58d3c371105d8119aa0ebb8bcadfabdcf')]